# Childcare Costs in the U.S.
## A project by The Penny Hoarder

In [37]:
import weightedcalcs as wc

import pandas as pd
import numpy as np
from easymoney.money import EasyPeasy
%matplotlib inline
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Data Cleaning and Preparation
<p>Let's start with the data. We grabbed the 2011 - 2017 annual American Community Survey public use microsamples courtesy of the University of Minnesota.

In [5]:
acs_raw = pd.read_csv("/Users/alexmahadevan/Desktop/childcare/childcare-data.csv")

In [6]:
# Inspect the variables.
acs_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1588798 entries, 0 to 1588797
Data columns (total 27 columns):
YEAR           1588798 non-null int64
SERIAL         1588798 non-null int64
MONTH          1588798 non-null int64
CPSID          1588798 non-null int64
ASECFLAG       1588798 non-null int64
HFLAG          199556 non-null float64
ASECWTH        1588798 non-null float64
STATEFIP       1588798 non-null int64
METRO          1588798 non-null int64
METAREA        1588798 non-null int64
COUNTY         1588798 non-null int64
STATECENSUS    1588798 non-null int64
PERNUM         1588798 non-null int64
CPSIDP         1588798 non-null int64
ASECWT         1588798 non-null float64
AGE            1588798 non-null int64
RACE           1588798 non-null int64
MARST          1588798 non-null int64
NCHILD         1588798 non-null int64
ELDCH          1588798 non-null int64
YNGCH          1588798 non-null int64
EMPSTAT        1588798 non-null int64
OCC            1588798 non-null int64
FTOTVAL 

In [10]:
# We need to set a weight for weightedcalcs to do its survey magic.
calc = wc.Calculator("ASECWT")

In [7]:
# Filter the dataset down to the respondents who have a single child under 6 years old.
# We do this to isolate a single child's costs, since there might be economies of scale for -
# multiple-child households.
acs_one_child = acs_raw[acs_raw["NCHILD"] == 1]
acs_toddler = acs_one_child[acs_one_child["ELDCH"] < 6]
acs_pay_childcare = acs_toddler[acs_toddler["SPMCHXPNS"] > 0]
acs_filter = acs_pay_childcare[acs_pay_childcare["PERNUM"] == 1]

In [9]:
# Now filter by year so we can compare across years. (Note: data on childcare costs starts in 2011)
acs_filter_2011 = acs_filter[acs_filter["YEAR"] == 2011]
acs_filter_2012 = acs_filter[acs_filter["YEAR"] == 2012]
acs_filter_2013 = acs_filter[acs_filter["YEAR"] == 2013]
acs_filter_2014 = acs_filter[acs_filter["YEAR"] == 2014]
acs_filter_2015 = acs_filter[acs_filter["YEAR"] == 2015]
acs_filter_2016 = acs_filter[acs_filter["YEAR"] == 2016]
acs_filter_2017 = acs_filter[acs_filter["YEAR"] == 2017]

In [26]:
# What were average childcare costs for a family with a single child under 6 in 2011?
avgchildcarecost2011 = calc.mean(acs_filter_2011, "SPMCHXPNS")

In [27]:
# What were average childcare costs for a family with a single child under 6 in 2017?
avgchildcarecost2017 = calc.mean(acs_filter_2017, "SPMCHXPNS")

In [32]:
# We need to convert those 2011 dollars to current using the CPI-U courtsy of the U.S. Bureau of Labor Staistics.
# Let's go ahead and caluclate the CPI for all years.
CPU2017 = 361
CPU2016 = 353.4
CPU2015 = 348.9
CPU2014 = 348.3
CPU2013 = 342.5
CPU2012 = 337.5
CPU2011 = 330.5
InflationCalc2011to2017 = CPU2017 / CPU2011
InflationCalc2012to2017 = CPU2017 / CPU2012
InflationCalc2013to2017 = CPU2017 / CPU2013
InflationCalc2014to2017 = CPU2017 / CPU2014
InflationCalc2015to2017 = CPU2017 / CPU2015
InflationCalc2016to2017 = CPU2017 / CPU2016

In [33]:
# Let's adjust it.
adjavgchildcarecost2011 = avgchildcarecost2011 * InflationCalc2011to2017

In [34]:
# How much more expensive has childcare for a child under 6 gotten in the last 6 years?
(avgchildcarecost2017 - adjavgchildcarecost2011) / adjavgchildcarecost2011

0.1704328778978211

In [45]:
# Has income kept up with it?
avghhincome2011 = calc.mean(acs_filter_2011, "FTOTVAL")
avghhincome2017 = calc.mean(acs_filter_2017, "FTOTVAL")

In [46]:
adjavghhincome2011 = avghhincome2011 * InflationCalc2011to2017

In [47]:
# How much has average income risen in the last 6 years?
(avghhincome2017 - adjavghhincome2011) / adjavghhincome2011

0.1475516899176262

In [ ]:
# The amount spent on childcare has outpaced the growth in income by three percentage points.